In [11]:
!pip install -q huggingface
!pip install -q transformers
!pip install -q pillow
!pip install -q loguru
!pip install -q pydantic
!pip install -q pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import requests
from PIL import Image
import torch

from transformers import OwlViTProcessor, OwlViTForObjectDetection

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
processor = OwlViTProcessor.from_pretrained("google/owlvit-large-patch14")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-large-patch14").to(device)

preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [5]:
model.eval()

OwlViTForObjectDetection(
  (owlvit): OwlViTModel(
    (text_model): OwlViTTextTransformer(
      (embeddings): OwlViTTextEmbeddings(
        (token_embedding): Embedding(49408, 768)
        (position_embedding): Embedding(16, 768)
      )
      (encoder): OwlViTEncoder(
        (layers): ModuleList(
          (0-11): 12 x OwlViTEncoderLayer(
            (self_attn): OwlViTAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): OwlViTMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_

In [6]:
model.device

device(type='cuda', index=0)

In [7]:

import time
import torch
from utils.protocols import OwlVit
from utils.utils import convert_model_detection
from utils.video import read_video
from utils.utils import get_file_name, get_gpu_name
from loguru import logger
from datetime import datetime

MODEL= "owlvit-large-p14-hf"

text = "face"
BASE_DIR = "experiments/owlvit"

def process_video(video, frames=[]):
    start_time = datetime.now().isoformat()
    start = time.time()
    
    results = {}
    for frame_id, frame in read_video(video):
        image = Image.fromarray(frame.astype("uint8"))
        inputs = processor(text=[[text]], images=image, return_tensors="pt")

        inputs = processor(images=image, text=text, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        target_sizes = torch.Tensor([image.size[::-1]])
        detections = processor.post_process_object_detection(outputs=outputs, threshold=0.1, target_sizes=target_sizes)
        results[frame_id] = convert_model_detection(detections[0])

    end = time.time()
    end_time = datetime.now().isoformat()
    n_frames = frame_id+1 if not frames else len(frames)
    
    exp =  OwlVit(
        model=MODEL,
        gpu=get_gpu_name(),
        file=video,
        prompt=text,
        frames=None if not frames else frames,
        n_frames=n_frames,
        processing_time=end-start,
        fps=n_frames/(end-start),
        data=results,
        start_time=start_time,
        end_time=end_time,
        filename=get_file_name(BASE_DIR, start, MODEL, video)
    )
    exp.save()
    exp.log()
    torch.cuda.empty_cache()
    return exp

In [8]:
results = []
for i in range(5):
    vid_hd = process_video("data/720.mp4")
    results.append(vid_hd)
    vid_fhd = process_video("data/1080.mp4")
    results.append(vid_fhd)

2024-06-11 11:52:50.058 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=100.6524441242218 | fps=2.8116555187743986
2024-06-11 11:54:36.764 | INFO     | utils.protocols:log:88 - data/1080.mp4 | frames=283 | delta=106.67646718025208 | fps=2.6528812537615503
2024-06-11 11:56:16.699 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=99.90661096572876 | fps=2.832645380164865
2024-06-11 11:58:03.525 | INFO     | utils.protocols:log:88 - data/1080.mp4 | frames=283 | delta=106.79778003692627 | fps=2.6498678146881915
2024-06-11 11:59:44.090 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=100.53894138336182 | fps=2.8148297177797184
2024-06-11 12:01:31.067 | INFO     | utils.protocols:log:88 - data/1080.mp4 | frames=283 | delta=106.92804145812988 | fps=2.6466397040556955
2024-06-11 12:03:11.185 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=100.086674451828 | fps=2.8275492371984914
2024-06-11 12:04:57.70

In [9]:
columns = results[0].columns
rows = [result.row for result in results]

In [12]:
import pandas as pd

df = pd.DataFrame(rows, columns=columns)
df.head(1)

,model,gpu,file,batch_size,n_frames,processing_time,fps,start_time,end_time,result_file,prompt,data
0,owlvit-large-p14-hf,NVIDIA L40,data/720.mp4,1,283,100.652444,2.811656,2024-06-11T11:51:09.388296,2024-06-11T11:52:50.040767,experiments/owlvit/17181066693883045-owlvit-la...,face,"{0: [box=[367.13006591796875, 66.2854843139648..."
1,owlvit-large-p14-hf,NVIDIA L40,data/1080.mp4,1,283,106.676467,2.652881,2024-06-11T11:52:50.068282,2024-06-11T11:54:36.744778,experiments/owlvit/17181067700682912-owlvit-la...,face,"{0: [box=[548.7283935546875, 98.23286437988281..."
2,owlvit-large-p14-hf,NVIDIA L40,data/720.mp4,1,283,99.906611,2.832645,2024-06-11T11:54:36.773718,2024-06-11T11:56:16.680358,experiments/owlvit/17181068767737272-owlvit-la...,face,"{0: [box=[367.13006591796875, 66.2854843139648..."
3,owlvit-large-p14-hf,NVIDIA L40,data/1080.mp4,1,283,106.797780,2.649868,2024-06-11T11:56:16.709576,2024-06-11T11:58:03.507387,experiments/owlvit/17181069767095864-owlvit-la...,face,"{0: [box=[548.7283935546875, 98.23286437988281..."
4,owlvit-large-p14-hf,NVIDIA L40,data/720.mp4,1,283,100.538941,2.814830,2024-06-11T11:58:03.534198,2024-06-11T11:59:44.073168,experiments/owlvit/17181070835342078-owlvit-la...,face,"{0: [box=[367.13006591796875, 66.2854843139648..."


In [13]:
from datetime import datetime
now = datetime.now()
csv_file = f"__{MODEL}-{get_gpu_name().replace(' ','_')}-{now.day}-{now.hour}-{now.min}.csv"
df.to_csv(csv_file)